In [20]:
from sympy import symbols, cos, pi, sin, exp, lambdify
import numpy as np
import tensorflow as tf
from tensorflow import keras
np.random.seed(114)
tf.random.set_seed(514)
#tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
x = symbols('x')
y = symbols('y')
t = symbols('t')
e = exp(1)

u11 = x*x*((x-1)**2)*(1-y)*e**(0-t)
u12 = x*y*(6*x+y-3*x*y+2*x*x*y-4*x*x-2)*e**(0-t)
p1 = cos(pi*x)*sin(pi*y)*e**(0-t)
u21 = x*x*(x-1)**2*(1-y)*e**(0-t)
u22 = x*y*(6*x+y-3*x*y+2*x*x*y-4*x*x-2)*e**(0-t)
p2 = cos(pi*x)*sin(pi*y)*e**(0-t)

u11 = lambdify((x, y, t), u11,'tensorflow')
u12 = lambdify((x, y, t), u12,'tensorflow')
u21 = lambdify((x, y, t), u21,'tensorflow')
u22 = lambdify((x, y, t), u22,'tensorflow')
p1 = lambdify((x, y, t), p1,'tensorflow')
p2 = lambdify((x, y, t), p2,'tensorflow')
class OnePinn (keras.Sequential):
    def __init__(self, Layers, name=None):
        super(OnePinn, self).__init__(name=name)
        self.add(keras.Input(shape=(Layers[0],), dtype=tf.float64))
        for i in range(1, len(Layers) - 1):
            self.add(keras.layers.Dense(Layers[i], dtype=tf.float64, activation='tanh'))
        self.add(keras.layers.Dense(Layers[-1], dtype=tf.float64, activation=None, name="outputs"))
        self.optimizer = tf.keras.optimizers.Adam()
    @tf.function
    def u_t (self,inputs):
        x, y, t = inputs[:, 0:1], inputs[:, 1:2], inputs[:, 2:3]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(t)
            X = tf.stack([x[:,0], y[:,0], t[:,0]], axis=1)
            u = self(X)
        U_t = tape.gradient(u, t)
        del tape
        return U_t
    @tf.function
    def u_x (self,inputs):
        x, y, t = inputs[:, 0:1], inputs[:, 1:2], inputs[:, 2:3]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            X = tf.stack([x[:,0], y[:,0], t[:,0]], axis=1)
            u = self(X)
        U_x = tape.gradient(u, x)
        del tape
        return U_x
    @tf.function
    def u_xx (self,inputs):
        x, y, t = inputs[:, 0:1], inputs[:, 1:2], inputs[:, 2:3]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(x)
            X = tf.stack([x[:,0], y[:,0], t[:,0]], axis=1)
            u = self(X)
            U_x = tape.gradient(u, x)
        U_xx=tape.gradient(U_x, x)
        del tape
        return U_xx
    @tf.function
    def u_xy (self,inputs):
        x, y, t = inputs[:, 0:1], inputs[:, 1:2], inputs[:, 2:3]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch([x,y])
            X = tf.stack([x[:,0], y[:,0], t[:,0]], axis=1)
            u = self(X)
            U_x = tape.gradient(u, x)
        U_xy=tape.gradient(U_x, y)
        del tape
        return U_xy
    @tf.function
    def u_yy (self,inputs):
        x, y, t = inputs[:, 0:1], inputs[:, 1:2], inputs[:, 2:3]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(y)
            X = tf.stack([x[:,0], y[:,0], t[:,0]], axis=1)
            u = self(X)
            U_y = tape.gradient(u, y)
        U_yy=tape.gradient(U_y, y)
        del tape
        return U_yy
    @tf.function
    def u_y (self,inputs):
        x, y, t = inputs[:, 0:1], inputs[:, 1:2], inputs[:, 2:3]
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(y)
            X = tf.stack([x[:,0], y[:,0], t[:,0]], axis=1)
            u = self(X)
        U_t = tape.gradient(u, y)
        del tape
        return U_t
    @tf.function
    def loss_u (self,X_train,u_label):
          x, y, t = X_train[:, 0:1], X_train[:, 1:2], X_train[:, 2:3]
          up=self(X_train)
          loss_U = tf.reduce_mean(tf.square(u_label - up))
          return loss_U
class UltPinn(keras.Model):
    def __init__(self,model_u11,model_u12,model_u21,model_u22,model_p1,model_p2,name=None):
          super(UltPinn,self).__init__(name=name)
          self.m11 = model_u11 
          self.m12 = model_u12
          self.m21 = model_u21 
          self.m22 = model_u22
          self.m1 = model_p1 
          self.m2 = model_p2
          self.optimizer = tf.keras.optimizers.Adam()
    @tf.function
    def loss_PDE(self, X_train_1,X_train_2,f11,f12,f21,f22):
             loss_11=self.m11.u_t(X_train_1)+self.m11(X_train_1)*self.m11.u_x(X_train_1)+self.m12(X_train_1)*self.m12.u_x(X_train_1)\
        -2*self.m11.u_xx(X_train_1)+self.m1.u_x(X_train_1)-self.m12.u_xy(X_train_1)-f11
             loss_11 = tf.reduce_mean(tf.square(loss_11))
             loss_12=self.m12.u_t(X_train_1)+self.m11(X_train_1)*self.m11.u_y(X_train_1)+self.m12(X_train_1)*self.m12.u_y(X_train_1)\
        -2*self.m12.u_yy(X_train_1)+self.m1.u_y(X_train_1)-self.m11.u_xy(X_train_1)-self.m12.u_xx(X_train_1)-f12
             loss_12= tf.reduce_mean(tf.square(loss_12))
             loss_eq12=self.m11.u_x(X_train_1)+self.m12.u_y(X_train_1)
             loss_eq12= tf.reduce_mean(tf.square(loss_eq12))
             loss_21=self.m21.u_t(X_train_2)+self.m21(X_train_2)*self.m21.u_x(X_train_2)+self.m22(X_train_2)*self.m22.u_x(X_train_2)\
        -2*self.m21.u_xx(X_train_2)+self.m2.u_x(X_train_2)-self.m22.u_xy(X_train_2)-f21
             loss_21 = tf.reduce_mean(tf.square(loss_21))
             loss_22=self.m22.u_t(X_train_2)+self.m21(X_train_2)*self.m21.u_y(X_train_2)+self.m22(X_train_2)*self.m22.u_y(X_train_2)\
        -2*self.m22.u_yy(X_train_2)+self.m2.u_y(X_train_2)-self.m21.u_xy(X_train_2)-self.m22.u_xx(X_train_2)-f22
             loss_22= tf.reduce_mean(tf.square(loss_22))
             loss_eq22=self.m21.u_x(X_train_2)+self.m22.u_y(X_train_2)
             loss_eq22= tf.reduce_mean(tf.square(loss_eq22))
             return loss_11+loss_12+loss_eq12+loss_21+loss_22+loss_eq22
    @tf.function
    def loss_boundary(self, X_boundary_train_1,X_boundary_train_2):
        loss_b11=self.m11(X_boundary_train_1)
        loss_b11 = tf.reduce_mean(tf.square(loss_b11))
        loss_b12=self.m12(X_boundary_train_1)
        loss_b12 = tf.reduce_mean(tf.square(loss_b12))
        loss_b21=self.m21(X_boundary_train_2)
        loss_b21 = tf.reduce_mean(tf.square(loss_b21))
        loss_b22=self.m22(X_boundary_train_2)
        loss_b22 = tf.reduce_mean(tf.square(loss_b22))
        return  loss_b11+loss_b12+loss_b21+loss_b22
    @tf.function
    def loss_I(self, X_I_train):
        loss_I11=self.m11(X_I_train)-self.m21(X_I_train)
        loss_I11 = tf.reduce_mean(tf.square(loss_I11))
        loss_I12=self.m12(X_I_train)-self.m22(X_I_train)
        loss_I12 = tf.reduce_mean(tf.square(loss_I12))
        loss_I21=2*self.m12.u_y(X_I_train)-self.m1(X_I_train)-2*self.m22.u_y(X_I_train)+self.m2(X_I_train)
        loss_I21 = tf.reduce_mean(tf.square(loss_I21))
        loss_I22=self.m22.u_x(X_I_train)+self.m21.u_y(X_I_train)-self.m12.u_x(X_I_train)-self.m11.u_y(X_I_train)
        loss_I22 = tf.reduce_mean(tf.square(loss_I22))
        return  loss_I11+loss_I12+loss_I21+loss_I22
    @tf.function
    def onetrainingstep(self,X_train_1,X_train_2,X_boundary_train_1,X_boundary_train_2,X_I_train,u_labels,f11,f12,f21,f22):
        with tf.GradientTape(persistent=True) as tape:
            loss_labels= self.m11.loss_u(X_train_1,u_labels[0]) + self.m12.loss_u(X_train_1,u_labels[1])+self.m21.loss_u(X_train_2,u_labels[2]) + self.m12.loss_u(X_train_2,u_labels[3])+self.m1.loss_u(X_train_1,u_labels[4]) + self.m2.loss_u(X_train_2,u_labels[5])
            loss=loss_labels+self.loss_PDE(X_train_1,X_train_2,f11,f12,f21,f22)+self.loss_boundary(X_boundary_train_1,X_boundary_train_2)+self.loss_I( X_I_train)
            gradients_m11= tape.gradient(loss,self.m11.trainable_variables)
            gradients_m12= tape.gradient(loss,self.m12.trainable_variables)
            gradients_m21= tape.gradient(loss,self.m21.trainable_variables)
            gradients_m22= tape.gradient(loss,self.m22.trainable_variables)
            gradients_m1= tape.gradient(loss,self.m1.trainable_variables)
            gradients_m2= tape.gradient(loss,self.m2.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients_m11, self.m11.trainable_variables))
            self.optimizer.apply_gradients(zip(gradients_m12, self.m12.trainable_variables))
            self.optimizer.apply_gradients(zip(gradients_m21, self.m21.trainable_variables))
            self.optimizer.apply_gradients(zip(gradients_m22, self.m22.trainable_variables))
            self.optimizer.apply_gradients(zip(gradients_m1, self.m1.trainable_variables))
            self.optimizer.apply_gradients(zip(gradients_m2, self.m2.trainable_variables))
        del tape
        return loss
    def train(self,X_train_1,X_train_2,X_boundary_train_1,X_boundary_train_2,X_I_train,u_labels,f11,f12,f21,f22,epochs=200):
        for epoch in tf.range(1,epochs+1):
            loss = self.onetrainingstep(X_train_1,X_train_2,X_boundary_train_1,X_boundary_train_2,X_I_train,u_labels,f11,f12,f21,f22)
            if epoch % 50 == 0:                
                tf.print(
                  "Training loss  at epoch ",epoch,":",loss)
    def predict(self, X_pre_1,X_pre_2,true_labels):
        u11_pred = self.m11(X_pre_1)
        u12_pred = self.m12(X_pre_1)
        p1_pred = self.m1(X_pre_1)
        u21_pred = self.m21(X_pre_2)
        u22_pred = self.m22(X_pre_2)
        p2_pred = self.m2(X_pre_2)
        u11_error = u11_pred - true_labels[0]
        u12_error = u12_pred - true_labels[1]
        p1_error = p1_pred - true_labels[4]
        u21_error = u21_pred - true_labels[2]
        u22_error = u22_pred - true_labels[3]
        p2_error = p2_pred - true_labels[5]
        mse_u11 = tf.reduce_mean(tf.square(u11_error))
        mse_u12 = tf.reduce_mean(tf.square(u12_error))
        mse_p1 = tf.reduce_mean(tf.square(p1_error))
        mse_u21 = tf.reduce_mean(tf.square(u21_error))
        mse_u22 = tf.reduce_mean(tf.square(u22_error))
        mse_p2 = tf.reduce_mean(tf.square(p2_error))
        return  (u11_pred, u12_pred, p1_pred, u21_pred, u22_pred, p2_pred), (u11_error, u12_error, p1_error, u21_error, u22_error, p2_error), (mse_u11, mse_u12, mse_p1, mse_u21, mse_u22, mse_p2)
layers = [3, 30,30 ,1]  
u11pinn = OnePinn(layers)
u12pinn = OnePinn(layers)
p1pinn = OnePinn(layers)
u21pinn = OnePinn(layers)
u22pinn = OnePinn(layers)
p2pinn = OnePinn(layers)
finpinn=UltPinn(u11pinn,u12pinn,u21pinn,u22pinn,p1pinn,p2pinn)
def generate_data(num):
    num_points = num
    num_jiedian = int(num/3)
    num_zhongdian=num-num_jiedian
    x_min, x_max = 0, 1
    y_1_min, y_1_max = 0, 1
    t_min, t_max = 0, 1
    y_2_min, y_2_max = -1,0

    x_train = np.random.uniform(x_min, x_max, (num_points, 1))
    y_1_train = np.random.uniform(y_1_min, y_1_max, (num_points, 1))
    y_2_train = np.random.uniform(y_2_min, y_2_max, (num_points, 1))
    t_train = np.random.uniform(t_min, t_max, (num_points, 1))
    x_train = tf.convert_to_tensor(x_train, dtype=tf.float64)
    y_1_train= tf.convert_to_tensor(y_1_train, dtype=tf.float64)
    y_2_train = tf.convert_to_tensor(y_2_train, dtype=tf.float64)
    t_train  = tf.convert_to_tensor(t_train , dtype=tf.float64)

    u11_train = u11(x_train, y_1_train, t_train)
    u12_train = u12(x_train, y_1_train, t_train)
    p1_train = p1(x_train, y_1_train, t_train)
    u21_train = u21(x_train, y_2_train, t_train)
    u22_train = u22(x_train, y_2_train, t_train)
    p2_train = p2(x_train, y_2_train, t_train)
    x_, y_1, t_ = x_train, y_1_train, t_train
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x_,y_1,t_])
        u_11=u11(x_,y_1,t_)
        u_12=u12(x_,y_1,t_)
        p_1=p1(x_,y_1,t_)
        u11x=tape.gradient( u_11, x_)
        u11y=tape.gradient(u_11, y_1)
        u11t=tape.gradient(u_11, t_)
        u12x=tape.gradient(u_12, x_)
        u12y=tape.gradient(u_12, y_1)
        u12t=tape.gradient(u_12, t_)
        p1x=tape.gradient( p_1, x_)
        p1y=tape.gradient( p_1, y_1)
    u11xx=tape.gradient(u11x,x_)
    u11xy=tape.gradient(u11x,y_1)
    u12xx=tape.gradient(u12x, x_)
    u12xy=tape.gradient(u12x, y_1)
    u12yy=tape.gradient(u12y, y_1)
    u11yy=tape.gradient(u11y, y_1)
    del tape
    f11=u11t+u_11*u11x+u_12*u12x-2*u11xx+p1x-u12xy
    f12=u12t+u_11*u11y+u_12*u12y-u12xx-u11xy-2*u12yy+p1y
    x_, y_2, t_ = x_train, y_2_train, t_train
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x_,y_2,t_])
        u_21=u21(x_,y_2,t_)
        u_22=u22(x_,y_2,t_)
        p_2=p2(x_,y_2,t_)
        u21x=tape.gradient(u_21, x_)
        u21y=tape.gradient(u_21, y_2)
        u21t=tape.gradient(u_21, t_)
        u22x=tape.gradient(u_22, x_)
        u22y=tape.gradient(u_22, y_2)
        u22t=tape.gradient(u_22, t_)
        p2x=tape.gradient( p_2, x_)
        p2y=tape.gradient( p_2, y_2)
    u21xx=tape.gradient(u21x,x_)
    u21xy=tape.gradient(u21x,y_2)
    u22xx=tape.gradient(u22x, x_)
    u22xy=tape.gradient(u22x, y_2)
    u22yy=tape.gradient(u22y, y_2)
    u21yy=tape.gradient(u21y, y_2)
    del tape
    
    f21=u21t+u_21*u21x+u_22*u22x-2*u21xx+p2x-u22xy
    f22=u22t+u_21*u21y+u_22*u22y-u22xx-u21xy-2*u22yy+p2y
    
    X_train_1 = np.concatenate([x_train, y_1_train, t_train], axis=1)
    X_train_2 = np.concatenate([x_train, y_2_train, t_train], axis=1)
    X_boundary_train_1_1=np.concatenate([np.zeros_like(x_train[0:num_jiedian,:]), y_1_train[0:num_jiedian,:], t_train[0:num_jiedian,:]],axis=1)
    X_boundary_train_1_2=np.concatenate([x_train[num_jiedian:num_zhongdian,:], np.ones_like(y_1_train[num_jiedian:num_zhongdian,:]), t_train[num_jiedian:num_zhongdian,:]],axis=1)
    X_boundary_train_1_3=np.concatenate([np.ones_like(x_train[num_zhongdian:num_points,:]), y_1_train[num_zhongdian:num_points,:], t_train[num_zhongdian:num_points,:]],axis=1)
    X_boundary_train_1=np.concatenate([X_boundary_train_1_1, X_boundary_train_1_2, X_boundary_train_1_3], axis=0)
    X_boundary_train_2_1=np.concatenate([np.zeros_like(x_train[0:num_jiedian,:]), y_2_train[0:num_jiedian,:], t_train[0:num_jiedian,:]],axis=1)
    X_boundary_train_2_2=np.concatenate([x_train[num_jiedian:num_zhongdian,:], np.ones_like(y_2_train[num_jiedian:num_zhongdian,:]), t_train[num_jiedian:num_zhongdian,:]],axis=1)
    X_boundary_train_2_3=np.concatenate([np.ones_like(x_train[num_zhongdian:num_points,:]), y_2_train[num_zhongdian:num_points,:], t_train[num_zhongdian:num_points,:]],axis=1)
    X_boundary_train_2=np.concatenate([X_boundary_train_2_1, X_boundary_train_2_2, X_boundary_train_2_3], axis=0)
    X_I_train=np.concatenate([x_train,np.zeros_like(y_1_train), t_train], axis=1)

    u_labels = [u11_train, u12_train, u21_train, u22_train, p1_train, p2_train]
    return X_train_1,X_train_2,X_boundary_train_1,X_boundary_train_2,X_I_train,u_labels,f11,f12,f21,f22
X_train_1,X_train_2,X_boundary_train_1,X_boundary_train_2,X_I_train,u_labels,f11,f12,f21,f22= generate_data(900)
X_pre_1,X_pre_2,X_boundary_pre_1,X_boundary_pre_2,X_I_pre,u_pre_labels,f11_pre,f12_pre,f21_pre,f22_pre= generate_data(9000)
model=finpinn
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
model.train(X_train_1,X_train_2,X_boundary_train_1,X_boundary_train_2,X_I_train,u_labels,f11,f12,f21,f22,epochs=500)
predictions, errors, mse = model.predict(X_pre_1,X_pre_2, u_pre_labels)
print("MSE for u11:", mse[0])
print("MSE for u12:", mse[1])
print("MSE for p1:", mse[2])
print("MSE for u21:", mse[3])
print("MSE for u22:", mse[4])
print("MSE for p2:", mse[5])
u11_pred, u12_pred, p1_pred, u21_pred, u22_pred, p2_pred = predictions
un11=u11_pred.numpy().reshape(X_pre_1.shape[0])
un12=u12_pred.numpy().reshape(X_pre_1.shape[0])
un21=u21_pred.numpy().reshape(X_pre_1.shape[0])
un22=u22_pred.numpy().reshape(X_pre_1.shape[0])
pn1=p1_pred.numpy().reshape(X_pre_1.shape[0])
pn2=p2_pred.numpy().reshape(X_pre_1.shape[0])
file_path ="D:\Desktop\graduate paper\ fluid pinn data_pred.npz"
predictions = {
    "u11_pred": un11,
    "u12_pred": un12,
    "p1_pred": pn1,
    "u21_pred": un21,
    "u22_pred": un22,
    "p2_pred": pn2,
    "x_pred_1":X_pre_1,
    "x_pred_2":X_pre_2
}

#np.savez(file_path, **predictions)


Training loss  at epoch  50 : 7.6256212049139513
Training loss  at epoch  100 : 1.94689417396932
Training loss  at epoch  150 : 1.2155763043155929
Training loss  at epoch  200 : 0.89821456373249076
Training loss  at epoch  250 : 0.70014564673953361
Training loss  at epoch  300 : 0.57957263232417733
Training loss  at epoch  350 : 0.46591357814455109
Training loss  at epoch  400 : 0.34192447553155003
Training loss  at epoch  450 : 0.27554337933271567
Training loss  at epoch  500 : 0.22914960831298575
MSE for u11: tf.Tensor(0.0005137321614779525, shape=(), dtype=float64)
MSE for u12: tf.Tensor(0.0015110669638538207, shape=(), dtype=float64)
MSE for p1: tf.Tensor(0.01473550377953617, shape=(), dtype=float64)
MSE for u21: tf.Tensor(0.0023282452622219204, shape=(), dtype=float64)
MSE for u22: tf.Tensor(0.005026843068849653, shape=(), dtype=float64)
MSE for p2: tf.Tensor(0.025085488224066522, shape=(), dtype=float64)


[[0.0869561  1.37692302]
 [0.55594247 0.56314228]
 [0.15463884 1.63497782]
 [0.54329871 0.22564182]
 [0.68710861 1.98386219]
 [0.62808373 1.64575373]
 [0.74657278 1.91139251]
 [0.32032524 1.01598107]
 [0.94465988 1.24513011]
 [0.24447369 0.76357269]
 [0.0869561  1.37692302]
 [0.55594247 0.56314228]
 [0.15463884 1.63497782]
 [0.54329871 0.22564182]
 [0.68710861 1.98386219]
 [0.62808373 1.64575373]
 [0.74657278 1.91139251]
 [0.32032524 1.01598107]
 [0.94465988 1.24513011]
 [0.24447369 0.76357269]]
